In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit

In [2]:
mks = pd.read_csv("MKS.csv")

In [3]:
#data preprocessing 
#remove NaN values
nans = mks[mks['Close'].isnull()]
mks.drop(nans.index, inplace=True) 

In [4]:
# Initialize a data frames to hold the Features and the value
X_len = 900
days = 10

Χ_columns = []
for j in range(days):
    Χ_columns.append('i+%s' % str(j))
   
X = pd.DataFrame(index=range(X_len),columns = Χ_columns)
y = pd.DataFrame(index=range(X_len), columns = ['Close'])
for i in range(X_len):
    for j in range(days):
        X.iloc[i]['i+%s' % str(j)] = mks.iloc[i + j]['Close']
    y.iloc[i] = mks.iloc[(i + j + 1)]['Close']
v = pd.concat([X, y], axis=1)
v.head()

,i+0,i+1,i+2,i+3,i+4,i+5,i+6,i+7,i+8,i+9,Close
0,459.8,469.9,471.6,461.9,452.9,453.3,456,442,435.5,426.7,416.6
1,469.9,471.6,461.9,452.9,453.3,456,442,435.5,426.7,416.6,427.7
2,471.6,461.9,452.9,453.3,456,442,435.5,426.7,416.6,427.7,434.5
3,461.9,452.9,453.3,456,442,435.5,426.7,416.6,427.7,434.5,442.3
4,452.9,453.3,456,442,435.5,426.7,416.6,427.7,434.5,442.3,440.2


In [5]:
def split_train_test_set(X, y, test_size=0.2):  
    # Train-test split
    if len(X) != len(y):
        return "Error"
    split_index = int(len(X) * (1-test_size))
    X_train = X[:split_index]
    X_test = X[split_index:]
    y_train = y[:split_index]
    y_test = y[split_index:]
    
    return X_train, X_test, y_train, y_test

In [6]:
# Import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error

def rmsp(test, pred):
    return np.sqrt(np.mean(((test - pred)/test)**2)) * 100

def rmse(test, pred):
    return np.sqrt(((test - pred) ** 2).mean())

def print_metrics(test, pred):
    print('RMSE %f'%  rmse(test, pred))
    print('Root Mean Squared Percentage Error %f'% rmsp(test, pred))
    print('Mean Absolute Error: %f'% mean_absolute_error(test, pred))
    print('Explained Variance Score: %f'% explained_variance_score(test, pred))
    print('Mean Squared Error: %f'% mean_squared_error(test, pred))
    print('R2 score: %f'% r2_score(test, pred))
    print('')

def create_cv_sets(n_splits = 3, size = 300):
    cv_sets = []
    for index in range(n_splits):
        X_train, X_test, y_train, y_test = split_train_test_set(X[index:index + size], y[index:index + size])    
        cv_sets.append([X_train, X_test, y_train, y_test])
    return cv_sets 

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn import grid_search
from sklearn.model_selection import GridSearchCV
def fit_model(X, y):
    regressor = LinearRegression()
    parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
    grid = GridSearchCV(regressor, parameters)
    grid.fit(X_train, y_train)
    return grid.best_estimator_

C:\Users\ppnevmatikatos\AppData\Local\Programs\Python\Python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\ppnevmatikatos\AppData\Local\Programs\Python\Python35\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [17]:
regressors = []
for X_train, X_test, y_train, y_test in create_cv_sets():
    regressor = fit_model(X_train, y_train)
    regressors.append(regressor)
    pred = regressor.predict(X_test)
    #print("X_train.head(): ", X_train.head())
    #print("X_train.tail(): ", X_train.tail())
    #print("Pred: ", pred[:5])
    #print("Test: ", y_test[:5]) 
    #print(y_test['Close'])
    print_metrics(y_test, pred)  

RMSE 6.500098
Root Mean Squared Percentage Error 1.156586
Mean Absolute Error: 4.574820
Explained Variance Score: 0.829707
Mean Squared Error: 42.251269
R2 score: 0.829559

RMSE 6.597446
Root Mean Squared Percentage Error 1.173894
Mean Absolute Error: 4.736517
Explained Variance Score: 0.811301
Mean Squared Error: 43.526294
R2 score: 0.811288

RMSE 6.518486
Root Mean Squared Percentage Error 1.157825
Mean Absolute Error: 4.657421
Explained Variance Score: 0.807487
Mean Squared Error: 42.490660
R2 score: 0.807339



In [18]:
regressors[0].coef_

array([[ 0.07465039, -0.01583566, -0.00959529, -0.05746968,  0.15631765,
        -0.12891805, -0.08020704,  0.07736399, -0.0873028 ,  1.07202876]])

In [10]:
from sklearn.ensemble import RandomForestRegressor

def random_forest(X_train, X_test, y_train, y_test):
    # Classify and predict
    reg = RandomForestRegressor()
    reg.fit(X_train, y_train.values.ravel())
    pred = reg.predict(X_test)
    #print("Pred: ", pred[:5])
    #print("Test: ", y_test[:5])
    print_metrics(y_test.values.ravel(), pred)
    return rmse(y_test.values.ravel(), pred)

In [11]:
for X_train, X_test, y_train, y_test in create_cv_sets():
    random_forest(X_train,X_test,y_train,y_test)

RMSE 29.463042
Root Mean Squared Percentage Error 5.106115
Mean Absolute Error: 25.430000
Explained Variance Score: -0.002346
Mean Squared Error: 868.070833
R2 score: -2.501781

RMSE 34.502370
Root Mean Squared Percentage Error 5.998945
Mean Absolute Error: 30.979167
Explained Variance Score: -0.024830
Mean Squared Error: 1190.413542
R2 score: -4.161150

RMSE 32.841459
Root Mean Squared Percentage Error 5.705767
Mean Absolute Error: 29.388333
Explained Variance Score: -0.006701
Mean Squared Error: 1078.561417
R2 score: -3.890420



In [12]:
from sklearn.svm import LinearSVR
def svr(X_train, X_test, y_train, y_test):
    # Classify and predict
    reg = LinearSVR()
    reg.fit(X_train, y_train.values.ravel())
    pred = reg.predict(X_test)

    #print("Pred: ", pred[:5])
    #print("Test: ", y_test[:5])

    print_metrics(y_test.values.ravel(), pred)
    return rmse(y_test.values.ravel(), pred)

In [13]:
for X_train, X_test, y_train, y_test in create_cv_sets(): 
    svr(X_train,X_test,y_train,y_test)

RMSE 7.677340
Root Mean Squared Percentage Error 1.372687
Mean Absolute Error: 6.382703
Explained Variance Score: 0.821995
Mean Squared Error: 58.941551
R2 score: 0.762231

RMSE 7.183030
Root Mean Squared Percentage Error 1.273596
Mean Absolute Error: 5.018855
Explained Variance Score: 0.806486
Mean Squared Error: 51.595920
R2 score: 0.776301

RMSE 8.350567
Root Mean Squared Percentage Error 1.474677
Mean Absolute Error: 6.196704
Explained Variance Score: 0.803099
Mean Squared Error: 69.731973
R2 score: 0.683821



In [14]:
last = np.array([310.5,312.5, 313.5,316.200012,316.299988,314.799988,316.100006,320.5,308.79998,308.700012]).reshape(1,-1)
pred = regressor.predict(last)
pred

array([[ 309.39884063]])

In [15]:
last = np.array([316.200012, 316.299988,314.799988,316.100006,320.5,308.799988,313.700012,310.80,318.30,324.00]).reshape(1,-1)
pred = regressor.predict(last)
pred


array([[ 326.34387598]])

In [23]:
from sklearn.externals import joblib
joblib.dump(regressors[0], 'prediction_model.pkl') 

['prediction_model.pkl']